In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import gc
!pip install segmentation-models
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

In [ ]:
train_Img_Id = []
train_class_Id = []
for i in os.listdir(trainImgPath):
    for j in range(1,5):
        train_Img_Id.append(i)
        train_class_Id.append(j)
train_Imgs = pd.DataFrame({'ImageId':train_Img_Id,'ClassId':train_class_Id})
train_Imgs.head(10)
 

In [ ]:
train_data = pd.merge(train_Imgs,data ,how='outer', on=['ImageId','ClassId']) 
train_data = train_data.fillna('') 
train_data

In [ ]:
train_data = pd.pivot_table(train_data, values='EncodedPixels', index='ImageId',columns='ClassId', aggfunc=np.sum).astype(str)
train_data = train_data.reset_index() # add Index column to one level with classID   
train_data.columns = ['ImageId','Defect_1','Defect_2','Defect_3','Defect_4'] 
has_defect = []
stratify = []
for index,row in train_data.iterrows():
    if row.Defect_1 or row.Defect_2 or row.Defect_3 or row.Defect_4: 
        has_defect.append(1)
    else:
        has_defect.append(0) 
    if row.Defect_1 != '':
        stratify.append(1)
    elif row.Defect_2 != '':
        stratify.append(2)
    elif row.Defect_3:
        stratify.append(3)
    elif row.Defect_4:
        stratify.append(4)
    else:
        stratify.append(0)
        
train_data["has_defect"] = has_defect 
train_data["stratify"] = stratify 

train_data.head(5) 

In [ ]:
WIDTH=288
HEIGHT=288
TRAINING_SIZE=7095 

In [ ]:
x_train, x_test = train_test_split(train_data, test_size = 0.1, stratify=train_data['stratify'], random_state=42)
x_train, x_val = train_test_split(x_train, test_size = 0.2, stratify = x_train['stratify'], random_state=42)
print(x_train.shape, x_val.shape, x_test.shape) 

In [ ]:
x_train_classification = x_train[['ImageId','has_defect']]
x_val_classification = x_val[['ImageId','has_defect']]
x_test_classification = x_test[['ImageId','has_defect']] 
print(x_train_classification.shape , x_val_classification.shape,x_test_classification.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale=1./255., shear_range=0.2, zoom_range=0.05, rotation_range=5,
                           width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_dataframe(
        dataframe=x_train_classification.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

valid_data_generator = test_datagen.flow_from_dataframe(
        dataframe=x_val_classification.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Dropout
from keras.models import Model, load_model 

In [ ]:
Classification_Model = keras.applications.xception.Xception(include_top = False, input_shape = (HEIGHT,WIDTH,3))

layer = Classification_Model.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(512, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(64, activation='relu')(layer)
predictions = Dense(1, activation='sigmoid')(layer)
model = Model(inputs=Classification_Model.input, outputs=predictions)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
Training = model.fit_generator(train_data_generator, validation_data = valid_data_generator, epochs = 30, verbose=1)

In [ ]:
def convert_to_mask(encoded_pixels):
    counts=[]
    mask=np.zeros((256*1600), dtype=np.uint8) #don't change this
    pre_mask=np.asarray([int(point) for point in encoded_pixels.split()])
    for index,count in enumerate(pre_mask):
        if(index%2!=0):
            counts.append(count)
    i=0
    for index,pixel in enumerate(pre_mask):
        if(index%2==0):
            if(i==len(counts)):
                break
            mask[pixel:pixel+counts[i]]=1
            i+=1
    mask=np.reshape(mask,(1600,256)) #don't change this
    mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
    return mask

In [ ]:
def generate_imgs(train_data):
    list_of_imgs=np.empty(( len(train_data) ,HEIGHT,WIDTH,3), dtype=np.uint8)
    list_of_masks=np.empty((len(train_data) ,HEIGHT,WIDTH), dtype=np.uint8)
    i=0
    j=0
    for index,row in train_data.iterrows():
        imgpath = trainImgPath + '/' + row.ImageId
        rle=row[1]
        rle=''.join(rle)
        mask=convert_to_mask(rle)
        y=np.zeros((WIDTH,HEIGHT))
        y=mask
        list_of_masks[i]=y
        img=cv2.imread(imgpath)
        img=cv2.resize(img,(HEIGHT,WIDTH))
        list_of_imgs[i]=img
        i+=1
        del y
        del mask
        del img
        gc.collect()
    return list_of_imgs,list_of_masks
    

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
train_img_defect_1 = x_train[x_train['Defect_1']!=''][['ImageId','Defect_1']]
val_img_defect_1 =  x_val[x_val['Defect_1']!=''][['ImageId','Defect_1']] 
test_img_defect_1 =  x_test[x_test['Defect_1']!=''][['ImageId','Defect_1']] 
    
train_img_defect_1

In [ ]:
x_tr_sgm , y_tr_sgm =  generate_imgs(train_img_defect_1)
x_val_sgm , y_val_sgm =  generate_imgs(val_img_defect_1) 
print(x_val_sgm)
print(y_val_sgm)


In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score 

network = 'resnet34'
preprocess = get_preprocessing(network) 

model = Unet(network, classes=1, activation='sigmoid', encoder_weights='imagenet') 
model.summary()

In [ ]:
train_datagen_segm = ImageDataGenerator(rescale=1./255., shear_range=0.2, zoom_range=0.05, rotation_range=5,
                           width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_datagen_segm = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen_segm.flow_from_dataframe(
        dataframe=train_img_defect_1.astype(str),
        directory=trainImgPath,
        x_col=x,
        y_col=y,
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

valid_data_generator = test_datagen_segm.flow_from_dataframe(
        dataframe=val_img_defect_1.astype(str),
        directory=trainImgPath,
        x_col=x_val_sgm,
        y_col=y_val_sgm,
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

In [ ]:
import segmentation_models as sm

In [ ]:
model.compile(optimizer='adam', loss=sm.losses.dice_loss,metrics=[dice_coef])


In [ ]:
train_defect_1 = model.fit_generator(train_data_generator, validation_data = valid_data_generator, epochs = 30, verbose=1 )


In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x_train,y_train)

In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
 
network = 'resnet34'
process_input = get_preprocessing(network)
xtrain = process_input(xtrain)
model = Unet(network,input_shape = (WIDTH, HEIGHT, 3),classes=4,activation='softmax')
model.compile('adam', loss='binary_crossentropy',metrics=[dice_coef])


In [ ]:
model.fit(
    x=xtrain,
    y=ytrain,
    batch_size=16,
    epochs=50, 
)